# SQLite3

In [1]:
import sqlite3

In [2]:
sqlite3.version

'2.6.0'

In [4]:
sqlite3.sqlite_version

'3.27.2'

In [8]:
#Database 연결(생성)
conn = sqlite3.connect('./SQL/dbtest')

In [9]:
cur = conn.cursor()        #cursor 객체 만들기.

In [10]:
type(cur)

sqlite3.Cursor

In [11]:
dir(cur)  
#cursor를 받아서 하는 방법이 제대로 된 방법.
# connection은 접속만을 위해. cursor는 그 DB의 쿼리문을 수행하는데 쓰임.(execute를 통해)

#  'execute',    : 하나의 SQL문
#  'executemany', : 여러 번 실행하는 SQL문
#  'executescript', : 여러 줄.(세미콜론 여러개.)
#  'fetchall',★      : 리스트의 형태로 결과 받아오기
#  'fetchmany',       : 지정한 만큼 결과 가져옴
#  'fetchone',★      : 결과 1개

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize']

In [16]:
tuple(cur.execute("select sqlite_version()"))

(('3.27.2',),)

In [14]:
cur.fetchall()                       #   execute -> fetchall    을 반복한다.

[('3.27.2',)]

In [17]:
sqlite3.complete_statement("select sqlite_version();")            #구문이 문법에 맞는지 체크해주는 아이

True

##  cur.execute("SQL")

In [19]:
cur.execute("CREATE TABLE people (name_last, age)")

OperationalError: table people already exists

In [20]:
who = "Yeltsin"
age = 72

In [25]:
cur.execute("INSERT INTO people VALUES(?,?)", (who,age))
cur.execute("SELECT * FROM people WHERE name_last:who and age=:age", {"who":who,"age":age})

print(cur.fetchone())

OperationalError: near ":who": syntax error

In [26]:
# cur.executemany('SQL', params)

sql = "INSERT INTO  people VALUES(?,?)"
curData = [('A',1),('B',2),('C',3)]

cur.executemany(sql, curData)

In [29]:
cur.execute("SELECT DISTINCT * FROM people;")

In [30]:
cur.fetchall()

[('Yeltsin', 72), ('A', 1), ('B', 2), ('C', 3)]

In [32]:
# 컬럼 정보
col_info = "SELECT * FROM sqlite_master"
cur.execute(col_info)
cur.fetchall()

[('table', 'people', 'people', 2, 'CREATE TABLE people (name_last, age)')]

In [36]:
cur.execute("""
    SELECT DISTINCT * 
    FROM people 
    WHERE name_last=:who AND age=:age
    """, {"who":who, "age":age})
cur.fetchall()

[('Yeltsin', 72)]

In [40]:
data = [("A",1), ("B", 1), ("C",1)]
cur.executemany("INSERT INTO people VALUES(?,?)",data)

In [41]:
dataDict = [{"who":"E", "age":1},{"who":"F","age":1}]
cur.executemany("INSERT INTO people VALUES(:who, :age)", dataDict)

In [42]:
cur.execute("SELECT DISTINCT * FROM people")
cur.fetchall()

[('Yeltsin', 72),
 ('A', 1),
 ('B', 2),
 ('C', 3),
 ('B', 1),
 ('C', 1),
 ('E', 1),
 ('F', 1)]

In [43]:
# cur.executescript('''SQL1; SQL2; ...''')     #여러개 SQL문 실행하는 용도

In [44]:
cur.executescript("""
    CREATE TABLE person (
        first_name text primary key,
        last_name text not null
    );
    
    INSERT INTO person VALUES('name', 'kim');
""")

In [46]:
cur.execute("SELECT * FROM person;")
print(cur.fetchall())

[('name', 'kim')]


## cur.fetchone()     하나만 쓰기

In [48]:
#cur.fetchmany()             # returning a list

In [49]:
conn.close()                 # connection 닫기

# 총 복습

## DDL

In [60]:
conn = sqlite3.connect('connect.db')
print("Opened database successfully")

Opened database successfully


In [107]:
cur = conn.cursor()
cur.execute('''
    CREATE TABLE COMPANY(
        ID INT PRIMARY KEY  NOT NULL,
        NAME           TEXT NOT NULL,
        AGE            INT  NOT NULL,
        ADDRESS        CHAR(50),
        SALARY         REAL);
''')
print("Table created")                #DB Browser for SQLIte 를 통해 확인

Table created


### DML

In [108]:
cur.execute("INSERT INTO company (ID,NAME,AGE,ADDRESS,SALARY) VALUES(1,'Paul',32,'California',20000.00)");

In [111]:
cur.lastrowid, conn.total_changes

(1, 13)

In [113]:
conn.commit()

In [116]:
cur.execute("""
    INSERT INTO company (ID,NAME,AGE,ADDRESS,SALARY) 
    VALUES(:id,:name,:age,:address,:salary)""", 
    {'id':2,'name':'Allen','age':25,'address':'Texas','salary':15000.00});

In [117]:
cur.lastrowid, conn.total_changes

(2, 14)

In [118]:
conn.commit()

In [119]:
data = [(3, 'Teddy', 23, 'Norway',200000.00),(4, 'Mark',25, 'Rich-Mond',65000.00)]

In [126]:
data1 = [(7, 'Teddy', 23, 'Norway',200000.00),(8, 'Mark',25, 'Rich-Mond',65000.00)]

In [127]:
cur.executemany("INSERT INTO company (ID,NAME,AGE,ADDRESS,SALARY) VALUES(?,?,?,?,?)", data1);

In [128]:
cur.lastrowid, conn.total_changes

(2, 20)

In [129]:
conn.commit()

In [130]:
cur.executescript("""
    INSERT INTO company VALUES(9, 'Mark1',25,'Rich-Mond',65000.00);
    
    INSERT INTO company VALUES(10, 'Mark2',25,'Rich-Mond',65000.00);
""");

In [131]:
cur.lastrowid, conn.total_changes           #executescript는 commit 안해도 반영이 됨.

(2, 22)

In [73]:
conn.commit()               #commit의 유무, Database의 실제 반영 단계

In [135]:
cur.execute("SELECT * FROM company")

In [136]:
for row in cur:
    print(row)

(1, 'Paul', 32, 'California', 20000.0)
(2, 'Allen', 25, 'Texas', 15000.0)
(3, 'Teddy', 23, 'Norway', 200000.0)
(4, 'Mark', 25, 'Rich-Mond', 65000.0)
(5, 'Mark1', 25, 'Rich-Mond', 65000.0)
(6, 'Mark2', 25, 'Rich-Mond', 65000.0)
(7, 'Teddy', 23, 'Norway', 200000.0)
(8, 'Mark', 25, 'Rich-Mond', 65000.0)
(9, 'Mark1', 25, 'Rich-Mond', 65000.0)
(10, 'Mark2', 25, 'Rich-Mond', 65000.0)


## SELECT

In [91]:
cursor = cur.execute("SELECT id, name, address, salary from COMPANY")

for row in cursor:
    print("ID = ",row[0])
    print("NAME = ",row[1])
    print("ADDRESS = ",row[2])
    print("SALARY = ",row[3], end='\n\n')
    
print("Operation done successfully")

ID =  1
NAME =  Paul
ADDRESS =  California
SALARY =  20000.0

ID =  2
NAME =  Allen
ADDRESS =  Texas
SALARY =  15000.0

ID =  3
NAME =  Teddy
ADDRESS =  Norway
SALARY =  200000.0

ID =  4
NAME =  Mark
ADDRESS =  Rich-Mond
SALARY =  65000.0

ID =  5
NAME =  Mark1
ADDRESS =  Rich-Mond
SALARY =  65000.0

ID =  6
NAME =  Mark2
ADDRESS =  Rich-Mond
SALARY =  65000.0

Operation done successfully


In [101]:
for i in range(4):
    print(type(row[i]))

<class 'int'>
<class 'str'>
<class 'str'>
<class 'float'>


In [137]:
cid = 1
cur.execute("UPDATE company SET SALARY = 25000.00 where ID = :id", {"id":cid})

In [138]:
conn.commit()

In [139]:
print("Total number of rows updated :", conn.total_changes)

Total number of rows updated : 23


In [140]:
cursor = cur.execute("SELECT * FROM company")
for row in cursor:
    print("ID = ", row[0])
    print("NAME = ", row[1])
    print("ADDRESS = ", row[2])
    print("SALARY = ", row[3], "\n")

ID =  1
NAME =  Paul
ADDRESS =  32
SALARY =  California 

ID =  2
NAME =  Allen
ADDRESS =  25
SALARY =  Texas 

ID =  3
NAME =  Teddy
ADDRESS =  23
SALARY =  Norway 

ID =  4
NAME =  Mark
ADDRESS =  25
SALARY =  Rich-Mond 

ID =  5
NAME =  Mark1
ADDRESS =  25
SALARY =  Rich-Mond 

ID =  6
NAME =  Mark2
ADDRESS =  25
SALARY =  Rich-Mond 

ID =  7
NAME =  Teddy
ADDRESS =  23
SALARY =  Norway 

ID =  8
NAME =  Mark
ADDRESS =  25
SALARY =  Rich-Mond 

ID =  9
NAME =  Mark1
ADDRESS =  25
SALARY =  Rich-Mond 

ID =  10
NAME =  Mark2
ADDRESS =  25
SALARY =  Rich-Mond 



## DELETE

In [141]:
cur.execute("DELETE FROM company WHERE ID=2;")
print("Total number of rows deleted :", conn.total_changes)

Total number of rows deleted : 24


In [143]:
conn.commit()

## 예제: Database 생성(열기), Cursor 생성 / 그 이후 -> PPT 참고

### Cursor를 무한히 열어둘 수 없다 -> Timeout당해서 닫힘 => 적당히!

# Dump
## txt 파일에 SQL문을 넣어놓는 덤프를 생성할 수 있다. -> import/export 방법 등등 존재

In [144]:
con = sqlite3.connect('music.db')

In [145]:
cur = con.cursor()

In [147]:
cur.executescript("""
    CREATE TABLE Artist(ID integer NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, NAME text);
    CREATE TABLE Genre(ID integer NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, NAME text);
    CREATE TABLE Album(ID integer NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, TITLE text, ARTIST_ID integer);
""")

In [148]:
cur.execute("""
    CREATE TABLE Track(
    ID integer NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, 
    TITLE text,
    LENGTH integer,
    RATING integer,
    COUNT integer,
    ALBUM_ID integer,
    GENRE_ID integer);
    """)

In [149]:
cur.executescript("""
    INSERT INTO ARTIST(NAME) VALUES('Led Zepplin'), ('AC/DC');
    INSERT INTO GENRE(NAME) VALUES('Rock'), ('Metal');
""")

In [185]:
con.commit()

In [184]:
cur.execute("""
    INSERT INTO Album(TITLE, ARTIST_ID)  
    VALUES ('Who Made Who', 2), ('IV', 1);
""")

In [173]:
cur.executescript("""
    INSERT INTO Track(TITLE, RATING, LENGTH, COUNT, ALBUM_ID, GENRE_ID)
    VALUES('Black Dog',5,297,0,2,1), ('Stairway',5,482,0,2,1),
    ('About to Rock',5,313,0,1,2),('Who Made Who',5,207,0,1,2);
""")

In [186]:
# Join

cur.execute("""
    SELECT Album.TITLE, Artist.NAME
    FROM Album
    JOIN Artist ON
    Album.ARTIST_ID = Artist.ID;
    """)

In [187]:
con.commit()

In [188]:
cur.execute("""
    SELECT Track.TITLE, Genre.NAME
    FROM Track
    JOIN Genre ON
    Track.GENRE_ID = Genre.ID;
""")

In [189]:
con.commit()

In [190]:
cur.execute("""
    SELECT Album.TITLE, Track.TITLE
    FROM Track
    JOIN Album ON
    Track.Album_ID = Album.ID;
""")

In [191]:
con.commit()

In [192]:
cur.execute("""
    SELECT Track.TITLE, Artist.NAME, Album.TITLE, Genre.NAME
    FROM Track
    JOIN Artist JOIN Album JOIN Genre ON
    Track.ALBUM_ID = Album.ID and
    Track.GENRE_ID = Genre.ID and
    Album.ARTIST_ID = Artist.ID;
""")

In [193]:
con.commit()